In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random
import math
import os

In [ ]:
df = pd.read_json("reviews_Video_Games_5.json",lines=True)
ratings = pd.read_csv("ratings_Video_Games.csv",header=None,names=['reviewerID','asin','rating','unixRatingTime'])

In [ ]:
df # VideoGame Reviews

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"[7, 10]","I got this version instead of the PS3 version,...",3,"awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""","[2, 2]",I had Dirt 2 on Xbox 360 and it was an okay ga...,4,DIRT 3,1308009600,"06 14, 2011"
...,...,...,...,...,...,...,...,...,...
231775,A1ICREREXO9J81,B00KHECZXO,Frustrated gamer,"[0, 1]",Funny people on here are rating sellers that a...,5,this is for rating the system not the seller,1405814400,"07 20, 2014"
231776,A3VVMIMMTYQV5F,B00KHECZXO,Johnny Saigon,"[8, 11]",All this is is the Deluxe 32GB Wii U with Mari...,1,Get the Other Bundle Which Includes Extra Whee...,1403308800,"06 21, 2014"
231777,A1DD4B97M4DUC5,B00KHECZXO,migit,"[62, 66]",The package should have more red on it and sho...,1,Fake bundle,1401321600,"05 29, 2014"
231778,A2Q9CNJ4T6ZK99,B00KHECZXO,"Philip Brown ""Philip & Chana""","[33, 36]",Can get this at Newegg for $329.00 and the pac...,1,Looks Like We Have Gougers Again.,1401667200,"06 2, 2014"


In [ ]:
ratings # VideoGame Ratings

,reviewerID,asin,rating,unixRatingTime
0,AB9S9279OZ3QO,0078764343,5.0,1373155200
1,A24SSUT5CSW8BH,0078764343,5.0,1377302400
2,AK3V0HEBJMQ7J,0078764343,4.0,1372896000
3,A10BECPH7W8HM7,043933702X,5.0,1404950400
4,A2PRV9OULX1TWP,043933702X,5.0,1386115200
...,...,...,...,...
1324748,A3HMVWAGUCNA1K,B00LGBJIQ4,5.0,1405987200
1324749,A14H2HEWFT7CLQ,B00LGBKSK4,5.0,1405900800
1324750,A2EHIE51TERF2C,B00LGBKSK4,5.0,1405900800
1324751,A39GDUJP1737E1,B00LI0VTS8,1.0,1405296000


In [ ]:
helpful = df['helpful'].apply(lambda x:pd.Series([x[0],x[1]],index=['HelpfulnessNumerator', 'HelpfulnessDenominator']))
df.drop(['helpful','overall'], axis=1, inplace=True)
df = pd.concat((df, helpful),axis=1)
reviews = pd.merge(df, ratings, on=["reviewerID","asin"])

In [ ]:
print("Unique users {}".format(len(np.unique(reviews.reviewerID))))
print("Unique video game products {}".format(len(np.unique(reviews.asin))))

Unique users 24303
Unique video game products 10672


In [ ]:
reviews.rating.value_counts()/len(reviews.rating)

5.0    0.518531
4.0    0.236448
3.0    0.121991
1.0    0.064082
2.0    0.058948
Name: rating, dtype: float64

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 5.8 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633718 sha256=692c9ff5c0ae28d5059732fb28a8efa2f5ecdd5dfaeb037081780cb98e4d8546
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
#Libraries and packages
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#Surprise library
from surprise import accuracy
from surprise.model_selection.validation import cross_validate
from surprise.model_selection import PredefinedKFold
from surprise.dataset import Dataset
from surprise.reader import Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise.model_selection import RandomizedSearchCV
from collections import defaultdict

In [ ]:
reviews.columns

Index(['reviewerID', 'asin', 'reviewerName', 'reviewText', 'summary',
       'unixReviewTime', 'reviewTime', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'rating', 'unixRatingTime'],
      dtype='object')

## Data Processing

Step 1: Drop the columns: Id, ProductId, ProfileName, Helpfulness Numerator, HelpfullnessDenominator, Time, Summary, Text

Step 2: Change the position of the UserID column and set a position 0. The Surprise reader reads the data in the data frame in the following order user (raw) ids, the item(raw) ids, and the ratings. So we need to change the position of the UserID column.

Step 3: Filter the users who have given **50** or more ratings. I have done this to improve the RMSE and MSE values and the complete dataset takes a longer time to evaluate and fit.

In [ ]:
reviews

,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,reviewTime,HelpfulnessNumerator,HelpfulnessDenominator,rating,unixRatingTime
0,A2HD75EMZR8QLN,0700099867,123,Installing the game was a struggle (because of...,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012",8,12,1.0,1341792000
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""",If you like rally cars get this game you will ...,Good rally game,1372550400,"06 30, 2013",0,0,4.0,1372550400
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""",1st shipment received a book instead of the ga...,Wrong key,1403913600,"06 28, 2014",0,0,1.0,1403913600
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"I got this version instead of the PS3 version,...","awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011",7,10,3.0,1315958400
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""",I had Dirt 2 on Xbox 360 and it was an okay ga...,DIRT 3,1308009600,"06 14, 2011",2,2,4.0,1308009600
...,...,...,...,...,...,...,...,...,...,...,...
231775,A1ICREREXO9J81,B00KHECZXO,Frustrated gamer,Funny people on here are rating sellers that a...,this is for rating the system not the seller,1405814400,"07 20, 2014",0,1,5.0,1405814400
231776,A3VVMIMMTYQV5F,B00KHECZXO,Johnny Saigon,All this is is the Deluxe 32GB Wii U with Mari...,Get the Other Bundle Which Includes Extra Whee...,1403308800,"06 21, 2014",8,11,1.0,1403308800
231777,A1DD4B97M4DUC5,B00KHECZXO,migit,The package should have more red on it and sho...,Fake bundle,1401321600,"05 29, 2014",62,66,1.0,1401321600
231778,A2Q9CNJ4T6ZK99,B00KHECZXO,"Philip Brown ""Philip & Chana""",Can get this at Newegg for $329.00 and the pac...,Looks Like We Have Gougers Again.,1401667200,"06 2, 2014",33,36,1.0,1401667200


In [ ]:
irrelevent_columns = ['reviewTime', 'unixReviewTime','unixRatingTime','summary', 'reviewText', 'reviewerName', 'HelpfulnessNumerator', 'HelpfulnessDenominator']

In [ ]:
reviews.drop(irrelevent_columns, axis=1, inplace=True)

In [ ]:
reviews

,reviewerID,asin,rating
0,A2HD75EMZR8QLN,0700099867,1.0
1,A3UR8NLLY1ZHCX,0700099867,4.0
2,A1INA0F5CWW3J4,0700099867,1.0
3,A1DLMTOTHQ4AST,0700099867,3.0
4,A361M14PU2GUEG,0700099867,4.0
...,...,...,...
231775,A1ICREREXO9J81,B00KHECZXO,5.0
231776,A3VVMIMMTYQV5F,B00KHECZXO,1.0
231777,A1DD4B97M4DUC5,B00KHECZXO,1.0
231778,A2Q9CNJ4T6ZK99,B00KHECZXO,1.0


In [ ]:
#change UserID position
reviews.rename(columns={"asin": "VideoGameId"},inplace=True)
first_column = reviews.pop('reviewerID')
reviews.insert(0, 'reviewerId', first_column)

In [ ]:
#filter the dataset
reviews_groupby_users_Ratings = reviews.groupby('reviewerId')['rating']

In [ ]:
reviews_groupby_users_Ratings = pd.DataFrame(reviews_groupby_users_Ratings.count())

In [ ]:
reviews_groupby_users_Ratings

,rating
reviewerId,
A00263941WP7WCIL7AKWL,6
A005481137I9SCAWEF7ON,27
A01588502N52TNG1BP7WG,6
A01803182IUSFNIFF5984,5
A023090719X7MTBCLM19B,6
...,...
AZZ1KF8RAO1BR,9
AZZHLZ8H23QD4,10
AZZKH219IDQGY,5


In [ ]:
reviews

,reviewerId,VideoGameId,rating
0,A2HD75EMZR8QLN,0700099867,1.0
1,A3UR8NLLY1ZHCX,0700099867,4.0
2,A1INA0F5CWW3J4,0700099867,1.0
3,A1DLMTOTHQ4AST,0700099867,3.0
4,A361M14PU2GUEG,0700099867,4.0
...,...,...,...
231775,A1ICREREXO9J81,B00KHECZXO,5.0
231776,A3VVMIMMTYQV5F,B00KHECZXO,1.0
231777,A1DD4B97M4DUC5,B00KHECZXO,1.0
231778,A2Q9CNJ4T6ZK99,B00KHECZXO,1.0


In [ ]:
reviews.shape

(231780, 3)

## Collaborative Filtering Recommender Model using SURPRISE Library

In [ ]:
def min_rating_filter_pandas(
    data,
    min_rating=1,
    filter_by="user",
    col_user="reviewerId",
    col_item="VideoGameId",
):
    split_by_column = col_user

    if min_rating < 1:
        raise ValueError("min_rating should be integer and larger than or equal to 1.")

    return data.groupby(split_by_column).filter(lambda x: len(x) >= min_rating)


In [ ]:
def split_pandas_data_with_ratios(data, ratios, seed=42, shuffle=False):
    if math.fsum(ratios) != 1.0:
        raise ValueError("The ratios have to sum to 1")

    split_index = np.cumsum(ratios).tolist()[:-1]

    if shuffle:
        data = data.sample(frac=1, random_state=seed)

    splits = np.split(data, [round(x * len(data)) for x in split_index])

    # Add split index (this makes splitting by group more efficient).
    for i in range(len(ratios)):
        splits[i]["split_index"] = i

    return splits

In [ ]:
def _do_stratification(
    data,
    ratio=0.75,
    min_rating=1,
    filter_by="reviewerId",
    is_random=True,
    seed=42,
    col_user="reviewerId",
    col_item="VideoGameId"
):
    multi_split = False
    ratio = ratio

    split_by_column = col_user if filter_by == "user" else col_item

    ratio = ratio if multi_split else [ratio, 1 - ratio]

    if min_rating > 1:
        data = min_rating_filter_pandas(
            data,
            min_rating=min_rating,
            filter_by=filter_by,
            col_user=col_user,
            col_item=col_item,
        )

    # Split by each group and aggregate splits together.
    splits = []

    # If it is for chronological splitting, the split will be performed in a random way.
    df_grouped = (
        data.groupby(split_by_column)
        if is_random is False
        else data.groupby(split_by_column)
    )

    for _, group in df_grouped:
        group_splits = split_pandas_data_with_ratios(
            group, ratio, shuffle=is_random, seed=seed
        )

        # Concatenate the list of split dataframes.
        concat_group_splits = pd.concat(group_splits)

        splits.append(concat_group_splits)

    # Concatenate splits for all the groups together.
    splits_all = pd.concat(splits)

    # Take split by split_index
    splits_list = [
        splits_all[splits_all["split_index"] == x].drop("split_index", axis=1)
        for x in range(len(ratio))
    ]

    return splits_list


In [ ]:
reviews

,reviewerId,VideoGameId,rating
0,A2HD75EMZR8QLN,0700099867,1.0
1,A3UR8NLLY1ZHCX,0700099867,4.0
2,A1INA0F5CWW3J4,0700099867,1.0
3,A1DLMTOTHQ4AST,0700099867,3.0
4,A361M14PU2GUEG,0700099867,4.0
...,...,...,...
231775,A1ICREREXO9J81,B00KHECZXO,5.0
231776,A3VVMIMMTYQV5F,B00KHECZXO,1.0
231777,A1DD4B97M4DUC5,B00KHECZXO,1.0
231778,A2Q9CNJ4T6ZK99,B00KHECZXO,1.0


In [ ]:
review_splitted = _do_stratification(
    reviews,
    ratio=0.8,
    min_rating=1,
    filter_by="reviewerId",
    is_random=True,
    seed=42,
    col_user="reviewerId",
    col_item="VideoGameId",
)

In [ ]:
review_train = review_splitted[0]
review_test = review_splitted[1]

In [ ]:
review_splitted_validation = _do_stratification(
    review_train,
    ratio=0.8,
    min_rating=1,
    filter_by="reviewerId",
    is_random=True,
    seed=221,
    col_user="reviewerId",
    col_item="VideoGameId",
)

In [ ]:
review_train = review_splitted_validation[0]
review_validation = review_splitted_validation[1]

In [ ]:
reader = Reader()
surprise_data = Dataset.load_from_df(reviews, reader)
review_train_surprise = Dataset.load_from_df(review_train, reader)
review_train_surprise_built = review_train_surprise.build_full_trainset()
review_test_surprise = Dataset.load_from_df(review_test, reader)
review_validation_surprise = Dataset.load_from_df(review_validation, reader)

In [ ]:
review_test_surprise.df

,reviewerId,VideoGameId,rating
7,AQTC623NCESZW,0700099867,1.0
10,A38NXTZUFB1O2K,0700099867,5.0
14,AFS6WERAP409A,0700099867,1.0
19,A29KT7UP7DLM1J,0700099867,1.0
6,AN3YYDZAS3O1Y,0700099867,5.0
...,...,...,...
231756,A1ZE1LPZHGG1QP,B00KAI3KW2,5.0
231760,A2YSY26ZVMNNV2,B00KAI3KW2,5.0
231765,A1XNZ7PCE45KK7,B00KAI3KW2,5.0
231752,A31ARER1KVHZB,B00KAI3KW2,5.0


In [ ]:
review_validation

,reviewerId,VideoGameId,rating
23,A1CB8HH9YJ2YZE,0700099867,3.0
5,A2UTRVO4FDCBH6,0700099867,4.0
13,A248LSBZT4P38V,0700099867,1.0
15,A14L115LBOB0A5,0700099867,1.0
29,A1PH1BOQGD7SCA,6050036071,5.0
...,...,...,...
231759,AQOFQWJYJRA5W,B00KAI3KW2,1.0
231772,A1M19BGTJ5PI1I,B00KAI3KW2,2.0
231763,A26EF0FFHL68HP,B00KAI3KW2,1.0
231750,A1ZR8AIM298DFT,B00KAI3KW2,1.0


In [ ]:
testset = [review_test_surprise.df.loc[i].to_list() for i in range(len(review_test_surprise.df))]

KeyError: ignored

In [ ]:
validationset = [review_validation_surprise.df.loc[i].to_list() for i in range(len(review_validation_surprise.df))]

In [ ]:
def get_top_n(predictions, n=5):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

class collab_filtering_based_recommender_model():
    def __init__(self, model, trainset, testset, data):
        self.model = model
        self.trainset = trainset
        self.testset = testset
        self.data = data
        self.pred_test = None
        self.recommendations = None
        self.top_n = None
        self.recommenddf = None

    def fit_and_predict(self):        
        print('**Fitting the train data**',)
        self.model.fit(self.trainset)       

       
        self.pred_test = self.model.test(self.testset)        
        rmse = round(accuracy.rmse(self.pred_test), 3)
          
        mse = round(accuracy.mse(self.pred_test), 3)
        
        
        self.top_n = get_top_n(self.pred_test)
        self.recommenddf = pd.DataFrame(columns=['reviewerId','VideoGameId','rating'])
        for item in self.top_n:
            subdf = pd.DataFrame(self.top_n[item], columns=['VideoGameId','rating'])
            subdf['reviewerId'] = item
            cols = subdf.columns.tolist()
            cols = cols[-1:] + cols[:-1]
            subdf = subdf[cols]        
            self.recommenddf = pd.concat([self.recommenddf, subdf], axis = 0)        
        return rmse
        
    def cross_validate(self):
        print('**Cross Validating the data**')
        traintest_cv = []
        for i in range(5):
          splitted = _do_stratification(
            self.trainset,
            ratio=0.75,
            min_rating=1,
            filter_by="reviewerId",
            is_random=True,
            seed=random.randint(1, 200),
            col_user="reviewerId",
            col_item="VideoGameId")
          traintest_cv.append(tuple(splitted[0], splitted[1]))
        pkf = PredefinedKFold = ()
        accuracy_list = []
        for trainset, testset in pkf.split(traintest_cv):
          SVD.fit(trainset)
          predictions = SVD.test(testset)
          accuracy_list.append(accuracy.rmse(predictions, verbose=True))
          print("Current CV rmse is: " + str(accuracy.rmse(predictions, verbose=True)))
        print('**Mean CV rmse is ' + str(sum(accuracy_list)/len(accuracy_list))  + '**')
        return sum(accuracy_list)/len(accuracy_list)

    def recommend(self, user_id, n=5):
        print('**Recommending top ' + str(n)+ ' products for userid : ' + user_id + '**')
        df = self.recommenddf[self.recommenddf['reviewerId'] == user_id].head(n)
        display(df)
        return df

In [ ]:
def find_best_model(model, parameters,data):
    clf = RandomizedSearchCV(model, parameters, n_jobs=-1, measures=['rmse','mse'])
    clf.fit(data)             
    print(clf.best_score)
    print(clf.best_params)
    print(clf.best_estimator)
    return clf
 
params= {
    "n_epochs": [5, 10, 15, 20],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
clf = find_best_model(SVD, params, review_train_surprise)

{'rmse': 1.0811620069164842, 'mse': 1.1689144340330784}
{'rmse': {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.6}, 'mse': {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.6}}
{'rmse': <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fdb4e265b10>, 'mse': <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fdb4e388350>}


In [ ]:
svd = clf.best_estimator['rmse']

In [ ]:
svd

In [ ]:
col_fil_svd = collab_filtering_based_recommender_model(svd, review_train_surprise_built, review_validation.items(), surprise_data)

In [ ]:
col_fil_svd.fit_and_predict()

**Fitting the train data**


ValueError: ignored

In [ ]:
col_fil_svd.cross_validate()

**Cross Validating the data**


AttributeError: ignored

In [ ]:
def cross_validate(model, trainset, validationset):
  

<generator object DataFrame.items at 0x7fdb62274650>

('reviewerId', 18        A10AYT89XSCE46
13        A248LSBZT4P38V
15        A14L115LBOB0A5
29        A1PH1BOQGD7SCA
34         AZGQXYB4TAODL
               ...      
231772    A1M19BGTJ5PI1I
231764    A3RPNJODORTLRB
231763    A26EF0FFHL68HP
231750    A1ZR8AIM298DFT
231778    A2Q9CNJ4T6ZK99
Name: reviewerId, Length: 30325, dtype: object)
('VideoGameId', 18        0700099867
13        0700099867
15        0700099867
29        6050036071
34        7100027950
             ...    
231772    B00KAI3KW2
231764    B00KAI3KW2
231763    B00KAI3KW2
231750    B00KAI3KW2
231778    B00KHECZXO
Name: VideoGameId, Length: 30325, dtype: object)
('rating', 18        1.0
13        1.0
15        1.0
29        5.0
34        1.0
         ... 
231772    2.0
231764    5.0
231763    1.0
231750    1.0
231778    1.0
Name: rating, Length: 30325, dtype: float64)
